In [13]:
# third-party
import pandas as pd
import scipy
import pickle

# local
from get_data import transform_overview_on_target, transform_overview_on_overall
from performance import get_breath_parameters_performance
from delay_performance import normality_test
from performance import bland_altman_plot, get_relative_errors


In [14]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]

### Activity-specific only RP analysis

In [15]:
# Transform overview from participant specific into all participants

overview_all_participants = transform_overview_on_target(overview, target="Activity")
overview_all_participants.keys()

dict_keys(['SNB', 'SGB', 'MIXB', 'STNB', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR'])

In [16]:
performance_all_participants = get_breath_parameters_performance(overview_all_participants, target="Activity")
performance_all_participants

,Activity,Sensor,MAE Ti (s),MRE Ti (%),MAE Te (s),MRE Te (%),MAE Tb (s),MRE Tb (%)
0,SNB,MAG,"0.17 , 0.28","8.43 , 13.26","0.18 , 0.33","7.80 , 11.88","0.16 , 0.32","3.40 , 5.43"
1,SNB,PZT,"0.37 , 0.50","15.62 , 22.19","0.38 , 0.53","12.94 , 18.33","0.29 , 0.45","4.64 , 7.01"
2,SGB,MAG,"0.81 , 0.64","17.37 , 14.85","0.69 , 0.66","11.85 , 11.26","0.64 , 0.67","5.57 , 5.62"
3,SGB,PZT,"1.57 , 1.30","35.82 , 30.03","1.64 , 1.31","26.41 , 20.92","0.84 , 0.97","6.97 , 7.78"
4,MIXB,MAG,"0.19 , 0.19","7.51 , 7.52","0.18 , 0.17","6.25 , 5.74","0.20 , 0.21","3.51 , 3.28"
5,MIXB,PZT,"0.56 , 0.72","17.27 , 22.19","0.64 , 0.87","16.58 , 25.04","0.65 , 0.96","8.14 , 11.90"
6,STNB,MAG,"0.16 , 0.24","7.30 , 9.85","0.19 , 0.29","7.46 , 10.80","0.13 , 0.22","2.48 , 2.69"
7,STNB,PZT,"0.29 , 0.44","10.53 , 15.44","0.35 , 0.56","10.87 , 18.98","0.26 , 0.48","4.02 , 7.54"
8,MCH,MAG,"0.31 , 0.36","14.31 , 14.77","0.33 , 0.36","15.89 , 15.80","0.33 , 0.42","7.13 , 8.29"
9,MCH,PZT,"0.63 , 0.60","28.61 , 24.60","0.67 , 0.64","30.65 , 27.90","0.62 , 0.65","13.03 , 13.55"


In [17]:
# mag = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].loc[:,["SSE Ti", "SSE Te", "SSE Tb"]].reset_index(drop=True) 
# pzt = performance_all_participants[performance_all_participants["Sensor"] == "PZT"].loc[:,["SSE Ti", "SSE Te", "SSE Tb"]].reset_index(drop=True)

# a = mag / pzt 
# a["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].reset_index(drop=True)["Activity"]
# a

### Overall RP analysis

In [18]:
# Transform overview from participant specific into all participants
overview_all = transform_overview_on_overall(overview)
overview_all.keys()

dict_keys(['MAG', 'Airflow', 'PZT'])

In [19]:
get_breath_parameters_performance(overview_all, target=None)

N=2814 breaths for MAG
N=2299 breaths for PZT


,Parameter,Sensor,Abs. error (s),Rel. error (%),R^2,Bias (s),Variability (s)
0,tI,MAG,"0.29 , 0.36","13.91 , 15.85",0.90,-0.007,0.463
1,tI,PZT,"0.57 , 0.59","25.75 , 23.83",0.89,0.000,0.820
2,tE,MAG,"0.30 , 0.37","13.82 , 15.30",0.92,0.014,0.477
3,tE,PZT,"0.59 , 0.61","26.03 , 24.12",0.89,-0.002,0.847
4,tB,MAG,"0.26 , 0.38","5.45 , 7.08",0.97,0.004,0.455
5,tB,PZT,"0.48 , 0.59","10.16 , 11.64",0.96,0.010,0.759


In [20]:
import plotly.graph_objects as go

for metric in ["tI", "tE", "tB"]:
    rel_error_mag = get_relative_errors(overview_all["MAG"][f"{metric} airflow (s)"], overview_all["MAG"][f"{metric} (s)"])
    rel_error_pzt = get_relative_errors(overview_all["PZT"][f"{metric} airflow (s)"], overview_all["PZT"][f"{metric} (s)"])

    print(f"{metric}: {scipy.stats.ttest_ind(rel_error_mag, rel_error_pzt)}")

    fig = go.Figure()
    fig.add_trace(go.Violin(y=rel_error_mag, name="MAG"))
    fig.add_trace(go.Violin(y=rel_error_pzt, name="PZT"))

    # fig.show()

    # normality_test(rel_error_mag, sensor="MAG", type=f"relative error {metric}")
    # normality_test(rel_error_pzt, sensor="PZT", type=f"relative error {metric}")


tI: TtestResult(statistic=-0.482544313709333, pvalue=0.6294390028407708, df=5368.0)
tE: TtestResult(statistic=-1.3274146918709155, pvalue=0.18442822433340325, df=5346.0)
tB: TtestResult(statistic=0.12500856223929396, pvalue=0.9005216881944031, df=5111.0)


In [21]:
import numpy as np
import plotly.graph_objects as go
from constants import CATEGORICAL_PALETTE

rel_error_all = get_relative_errors(overview_all["MAG"]["tB airflow (s)"], overview_all["MAG"]["tB (s)"])

fig = go.Figure()
categorical_palette = iter(CATEGORICAL_PALETTE)
fig.add_trace(go.Violin(y=rel_error_all,
                            #line_color=next(categorical_palette), 
                            name="overall"))

for activity in overview_all_participants.keys():

    overview_all_except_activity = transform_overview_on_overall(overview, ignore_key=activity)

    rel_error_all_except = get_relative_errors(overview_all_except_activity["MAG"]["tB airflow (s)"], overview_all_except_activity["MAG"]["tB (s)"])
    rel_error_by_activity_mag = get_relative_errors(overview_all_participants[activity]["MAG"]["tB airflow (s)"], overview_all_participants[activity]["MAG"]["tB (s)"])
    rel_error_by_activity_pzt = get_relative_errors(overview_all_participants[activity]["PZT"]["tB airflow (s)"], overview_all_participants[activity]["PZT"]["tB (s)"])
    # print(f"{activity}: {scipy.stats.ttest_ind(rel_error_all, rel_error_by_activity)}")
    print(f"{activity}: {scipy.stats.ttest_ind(rel_error_all_except, rel_error_by_activity_pzt)}")
    #normality_test(rel_error_by_activity, sensor="MAG", type="relative error by activity")
    
    # fig.add_trace(go.Violin(y=rel_error_by_activity,
    #                         #line_color=next(categorical_palette), 
    #                         name=activity))
#fig.show()


SNB: TtestResult(statistic=-0.18714941610861674, pvalue=0.8515572949038666, df=2749.0)
SGB: TtestResult(statistic=-0.22483868319720035, pvalue=0.8221212191065481, df=2793.0)
MIXB: TtestResult(statistic=0.430764240082546, pvalue=0.6666735751302737, df=2749.0)
STNB: TtestResult(statistic=0.8142658217252154, pvalue=0.41556362611774067, df=2733.0)
MCH: TtestResult(statistic=1.1325696715903875, pvalue=0.25749240310459615, df=2788.0)
SQT: TtestResult(statistic=-0.37550341778047647, pvalue=0.7073150131136632, df=2755.0)
AAL: TtestResult(statistic=1.0880273982569568, pvalue=0.2766777378446152, df=2768.0)
AAR: TtestResult(statistic=-1.4081976629194854, pvalue=0.1591843255935917, df=2781.0)
ALL: TtestResult(statistic=-0.3507462605018836, pvalue=0.7258052826722232, df=2781.0)
ALR: TtestResult(statistic=0.9069187661353708, pvalue=0.3645286133811413, df=2771.0)
UAL: TtestResult(statistic=0.3491135278966605, pvalue=0.7270304167479289, df=2791.0)
UAR: TtestResult(statistic=0.200953759862235, pvalue=0

In [22]:
overview_all_activities = transform_overview_on_target(overview, target="ID")
overview_all_activities.keys()

fig = go.Figure()
categorical_palette = iter(CATEGORICAL_PALETTE)
fig.add_trace(go.Violin(y=rel_error_all,
                            #line_color=next(categorical_palette), 
                            name="overall"))

# print(f"Overall: {np.mean(rel_error_all)*100:.2f} +/- {np.std(rel_error_all)*100:.2f}")

for id in overview_all_activities.keys():

    overview_all_except_id= transform_overview_on_overall(overview, ignore_key=id)

    rel_error_by_participant = get_relative_errors(overview_all_activities[id]["MAG"]["tB airflow (s)"], overview_all_activities[id]["MAG"]["tB (s)"])
    #print(f"{id}: {scipy.stats.ttest_ind(rel_error_all, rel_error_by_participant)}")
    #normality_test(rel_error_by_participant, sensor="MAG", type="relative error by id")
    fig.add_trace(go.Violin(y=rel_error_by_participant,
                            name=id))
# fig.show()
    

### Bias and variability analysis by activity

In [23]:
from performance import get_bias_variability

get_bias_variability(overview_all_participants, target="Activity", metric="tI", relative_error=False)

,Activity,Sensor,Bias (s),Variability (s)
0,SNB,MAG,0.039,0.331
1,SNB,PZT,0.005,0.626
2,SGB,MAG,0.340,0.985
3,SGB,PZT,-0.026,2.058
4,MIXB,MAG,-0.038,0.270
5,MIXB,PZT,-0.129,0.908
6,STNB,MAG,-0.041,0.285
7,STNB,PZT,-0.055,0.528
8,MCH,MAG,0.015,0.481
9,MCH,PZT,-0.146,0.864


In [24]:
for metric in ["tI", "tE", "tB"]:
        bland_altman_plot(overview_all["MAG"][f"{metric} (s)"], overview_all["MAG"][f"{metric} airflow (s)"], overview_all["PZT"][f"{metric} (s)"], overview_all["PZT"][f"{metric} airflow (s)"], metric)
